In [ ]:
# https://zhuanlan.zhihu.com/p/143621989

In [6]:
import torch
import cv2
import os
import glob
from torch.utils.data import Dataset
import random
import numpy as np


class Gopro_Loader(Dataset):
    def __init__(self, data_path):
        # 初始化函数，读取所有data_path下的图片
        self.data_path = data_path
        self.imgs_path = glob.glob(os.path.join(data_path, 'blur/*.png'))

    def augment(self, image, flipCode):
        # 使用cv2.flip进行数据增强，filpCode为1水平翻转，0垂直翻转，-1水平+垂直翻转
        flip = cv2.flip(image, flipCode)
        return flip
        
    def __getitem__(self, index):
        # 根据index读取图片
        image_path = self.imgs_path[index]
        # 根据image_path生成label_path
        label_path = image_path.replace('blur', 'sharp')
        # 读取训练图片和标签图片
        image = cv2.imread(image_path)
        label = cv2.imread(label_path)
        # 将数据转为单通道的图片
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
#         print (image.shape)
#         image = image.transpose((2, 0, 1))  # 调整通道的顺序
#         label = label.transpose((2, 0, 1))  # 调整通道的顺序
        print (image.shape)
        image = image.reshape(1, image.shape[0], image.shape[1])
        label = label.reshape(1, label.shape[0], label.shape[1])
        # 处理标签，将像素值为255的改为1
        if label.max() > 1:
            label = label / 255
        # 随机进行数据增强，为2时不做处理
#         flipCode = random.choice([-1, 0, 1, 2])
#         if flipCode != 2:
#             image = self.augment(image, flipCode)
#             label = self.augment(label, flipCode)
        return image, label

    def __len__(self):
        # 返回训练集大小
        return len(self.imgs_path)

In [8]:

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class Gopro_Loader(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        
        # 讀取所有影像的檔名
        self.imgs = glob.glob(os.path.join(root, 'blur/*.png'))

    def __getitem__(self, idx):
        # 讀取影像
        img_path = self.imgs[idx]
        img = Image.open(img_path)
        # 將影像轉換成3通道
        img = img.convert('RGB')
        
        # 讀取標籤
        label_path = img_path.replace("blur", "sharp")
        label = Image.open(label_path)
        
        if self.transform:
            img = self.transform(img)
            label = self.transform(label)

        return img, label

    def __len__(self):
        return len(self.imgs)

In [13]:
from torchvision import transforms
from torch.utils.data import DataLoader

# 定義數據變換
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# 創建數據集
train_set = Gopro_Loader(root="dataset_gopro/train", transform=transform)

# 創建數據加載器
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)

In [14]:
print("数据个数：", len(train_set))

数据个数： 2123


In [118]:
dataset = Gopro_Loader("dataset_gopro/train")

In [119]:
print("数据个数：", len(dataset))

数据个数： 2103


In [120]:
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                               batch_size=2, 
                                               shuffle=True)
for image, label in train_loader:
    print(image.shape)

(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torch.Size([2, 1, 720, 1280])
(720, 1280)
(720, 1280)
torc

KeyboardInterrupt: 

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = torch.tensor([x2.size()[2] - x1.size()[2]])
        diffX = torch.tensor([x2.size()[3] - x1.size()[3]])

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [37]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [34]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)  # 修改這行

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)  # 使用 nn.Conv2d 改變
        logits = self.outc(x)
        return logits

In [35]:
net = UNet(n_channels=3, n_classes=3)
print(net)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (4): BatchNorm2d(128, eps=1e-05, moment

In [41]:
from torch import optim
import torch.nn as nn
import torch

def train_net(net, device, data_path, epochs=40, batch_size=1, lr=0.00001):
    # 加载训练集
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    isbi_dataset = Gopro_Loader(root=data_path, transform=transform)
    train_loader = torch.utils.data.DataLoader(dataset=isbi_dataset,
                                               batch_size=batch_size, 
                                               shuffle=True)
    # 定义RMSprop算法
    optimizer = optim.RMSprop(net.parameters(), lr=lr, weight_decay=1e-8, momentum=0.9)
    # 定义Loss算法
    criterion = nn.BCEWithLogitsLoss()
    # best_loss统计，初始化为正无穷
    best_loss = float('inf')
    # 训练epochs次
    for epoch in range(epochs):
        # 训练模式
        net.train()
        # 按照batch_size开始训练
        for image, label in train_loader:
            optimizer.zero_grad()
            # 将数据拷贝到device中
            image = image.to(device=device, dtype=torch.float32)
            label = label.to(device=device, dtype=torch.float32)
            # 使用网络参数，输出预测结果
            pred = net(image)
            print (f'pred: {pred.shape}')
            # 將預測輸出的通道數改成3
#             pred = pred.expand(-1, 3, -1, -1)
#             print (f'pred: {pred.shape}')
            print (f'label: {label.shape}')
            # 计算loss
            loss = criterion(pred, label)
            print('Loss/train', loss.item())
            # 保存loss值最小的网络参数
            if loss < best_loss:
                best_loss = loss
                torch.save(net.state_dict(), 'best_model.pth')
            # 更新参数
            loss.backward()
            optimizer.step()

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 加载网络，图片单通道1，分类为1。
net = UNet(n_channels=3, n_classes=1)
# 将网络拷贝到deivce中
net.to(device=device)
# 指定训练集地址，开始训练
data_path = "dataset_gopro/train"
train_net(net, device, data_path)

C:\Users\zhenRong\AppData\Local\Temp\ipykernel_16016\1205209092.py:55: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
C:\Users\zhenRong\AppData\Local\Temp\ipykernel_16016\1205209092.py:56: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  diffY // 2, diffY - diffY // 2])


pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.7226229906082153
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6932510137557983
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6389408707618713
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6530129909515381
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6382372379302979
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6058496832847595
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6234734058380127
pred: torch.Size([1, 1, 256, 256])
pred: torch.S

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5411695241928101
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6240909695625305
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5631300806999207
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5710593461990356
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5586671233177185
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6014963388442993
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5979375243186951
pred: torch.Size([1, 1, 256, 256])
pred: torch.S

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5685248374938965
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5943425893783569
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6174084544181824
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5367035865783691
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5664463639259338
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6321530342102051
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5664293169975281
pred: torch.Size([1, 1, 256, 256])
pred: torch.S

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.706782341003418
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5655920505523682
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5727982521057129
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6052977442741394
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6012619733810425
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6002429723739624
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5576472878456116
pred: torch.Size([1, 1, 256, 256])
pred: torch.Si

...

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5221879482269287
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6347132921218872
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5148950815200806
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6243034601211548
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.49027085304260254
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5742251873016357
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5028396248817444
pred: torch.Size([1, 1, 256, 256])
pred: torch.

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5815649032592773
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5679925680160522
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5851470232009888
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5795364379882812
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.569786012172699
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6137561798095703
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5362896919250488
pred: torch.Size([1, 1, 256, 256])
pred: torch.Si

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6200107932090759
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5673355460166931
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6184301972389221
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5643736720085144
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5469076633453369
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5565125942230225
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5255666971206665
pred: torch.Size([1, 1, 256, 256])
pred: torch.S

pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6342924237251282
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6212310791015625
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5137945413589478
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5737751722335815
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6340106725692749
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.6287955641746521
pred: torch.Size([1, 1, 256, 256])
pred: torch.Size([1, 3, 256, 256])
label: torch.Size([1, 3, 256, 256])
Loss/train 0.5918339490890503
pred: torch.Size([1, 1, 256, 256])
pred: torch.S

KeyboardInterrupt: 

In [50]:

from torch.utils.data import DataLoader

# 选择设备，有cuda用cuda，没有就用cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 加载网络，图片三通道，分类为1。
net = UNet(n_channels=3, n_classes=1)
# 将网络拷贝到device中
net.to(device=device)
# 加载模型参数
net.load_state_dict(torch.load('best_model.pth', map_location=device))
# 测试模式
net.eval()

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
# 加载测试集数据
test_dataset = Gopro_Loader(root='dataset_gopro/test', transform=transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

# 遍历测试集数据并测试模型
for image, _ in test_loader:
    # 将数据拷贝到device中
    image = image.to(device=device, dtype=torch.float32)
    # 使用网络参数，输出预测结果
    pred = net(image)
    print (pred.shape)
    # 將預測輸出的通道數改成3
#     pred = pred.expand(-1, 3, -1, -1)
    # 将结果拷贝回cpu
    pred = pred.cpu()
    # 处理结果
#     pred[pred >= 0.5] = 255
#     pred[pred < 0.5] = 0
    # 将Tensor转换为NumPy数组
#     pred = pred.detach().numpy()[0][0].astype('uint8')
    pred = pred.detach().cpu().numpy()

    print (pred.shape)
    # 保存图片
    save_res_path = test_path.split('.')[0] + '_res.png'
    cv2.imwrite(save_res_path, pred[0].transpose(1, 2, 0).astype('uint8'))

#     cv2.imwrite(save_res_path, pred)


C:\Users\zhenRong\AppData\Local\Temp\ipykernel_16016\1205209092.py:55: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
C:\Users\zhenRong\AppData\Local\Temp\ipykernel_16016\1205209092.py:56: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  diffY // 2, diffY - diffY // 2])


torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1

torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1, 256, 256)
torch.Size([1, 1, 256, 256])
(1, 1